<h1>Расчет LTV с помощью Когортного Анализа</h1>

<h3>VOLODYMYR CHUBAI</h3>
https://www.linkedin.com/in/volodymyr-chubai/

## Сценарий проекта
Я Junior Data Analyst, работаю в команде аналитиков данных. Приложение - мобильная утилита для сканирования документов. Модель монетизации подписочная, есть пробный период 7 дней с дальнейшим переходом в оплату 4.99 USD в неделю. Есть выгрузка с базы данных по оформлениям подписок и оплат. Каждая строка представляет собой отдельное событие (либо оформление пробной подписки, либо оплата после завершения пробного периода).

## 1. ASK
Моя задача заключаэться в том, чтобы рассчитать текущий LTV юзера, используя когортный анализ (cohorting event - оформление пробного периода, когорта представляет собой кол-во возможных операций).

## 2. PREPARE
### Источник данных
Используемые данные были взяты из следующего набора данных: [test case data analyst](https://drive.google.com/file/d/1L8843A80r50_e-hU79XV_ssAfzgQVUDP/view) 

Данные доступны по ссылке и хранятся в 1 XLSX-файле.

### Собранные данные включают
`Product ID` - Идентификатор продукта (название подписки).

`Quantity` - количество, сколько подписок было оформлено.

`Is Trial Period` - Строка представляет собой отдельное событие, либо оформление пробной подписки: 'True', либо оплата после завершения пробного периода: 'False'.

`Purchase Date` - Дата оформления подписки (дата покупки).

`User ID` - Уникальный идентификатор пользователя.

## 3. PROCESS
### Подготовка рабочей среды
Я буду использовать Python для очистки, преобразования и визуализации данных. Установлены следующие библиотеки:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

### Импорт наборов данных

In [3]:
df = pd.read_excel('/Users/volodymyrchubay/Desktop/test case data analyst.xlsx')

### Просмотр данных

In [4]:
df.head(3)

,product_id,quantity,is_trial_period,purchase_date,user_id
0,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-19,9484222
1,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9422402
2,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,4997991


### Проверка данных

In [5]:
df.shape

(114200, 5)

In [6]:
df.tail()

,product_id,quantity,is_trial_period,purchase_date,user_id
114195,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8291100
114196,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-02,8236936
114197,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-03,8247910
114198,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-04,8294146
114199,1week_4.99_USD_subscription_7days_trial,1,True,2020-01-03,8274954


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114200 entries, 0 to 114199
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   product_id       114200 non-null  object        
 1   quantity         114200 non-null  int64         
 2   is_trial_period  114200 non-null  bool          
 3   purchase_date    114200 non-null  datetime64[ns]
 4   user_id          114200 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(2), object(1)
memory usage: 3.6+ MB


In [8]:
df.columns

Index(['product_id', 'quantity', 'is_trial_period', 'purchase_date',
       'user_id'],
      dtype='object')

### Очистка данных
Поиск нулевых значений

In [9]:
# количество отсутствующих данных в каждом столбце
df.isnull().sum()

product_id         0
quantity           0
is_trial_period    0
purchase_date      0
user_id            0
dtype: int64

#### Поиск и удаление дубликатов

In [10]:
dup_rows = df[df.duplicated()]
dup_rows = dup_rows.query('is_trial_period ==  True').sort_values(['user_id'], ascending=False)

In [11]:
df = df.drop_duplicates(keep='first')
# df.to_excel(r'/Users/volodymyrchubay/Desktop/df_no_dup.xlsx', index=False)

In [12]:
# в 35 пользователей значение True встречалось дважды что свидетельствует об ошибке
# данные строки были удалены, оставлен момент первого оформления пробной подписки
# также был добавлен столбец purchase_amount
# если пользователь оформил пробную подписку значение в строке равняется 0
df = pd.read_excel('/Users/volodymyrchubay/Desktop/clear_df.xlsx')
df.head(3)

,product_id,quantity,is_trial_period,purchase_date,user_id,purchase_amount
0,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-19,9484222,4.99
1,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-17,9422402,4.99
2,1week_4.99_USD_subscription_7days_trial,1,False,2020-02-18,4997991,4.99


### Просмотр общих показателей

In [13]:
query1  = df.query('is_trial_period ==  True') \
            .groupby('user_id') \
            .agg({'is_trial_period': 'count'}) \
            .sort_values(['is_trial_period'], ascending=False)

query1.count()
# количество уникальных пользователей что оформили пробную подписку

is_trial_period    62909
dtype: int64

In [14]:
quary2  = df.query('is_trial_period == False') \
            .groupby('user_id') \
            .agg({'purchase_amount': 'sum'}) \
            .sort_values(['purchase_amount'], ascending=False)

quary2.count()
# количество уникальных пользователей что оплатили подписку после завершения пробного периода 

purchase_amount    19546
dtype: int64

In [15]:
revenue = quary2['purchase_amount'].sum()
revenue

255348.28

## 4. ANALYSE
На этом этапе я буду выполнять некоторые расчеты, чтобы получить представление и выявить тенденции на основе данных. А также отвечу на поставленный мне вопрос и расчитаю текущий LTV юзера используя когорты.

In [16]:
cohort_df = df[['user_id', 'quantity', 'is_trial_period', 'purchase_date', 'purchase_amount']]
cohort_df = cohort_df.sort_values(by=['purchase_date'])
cohort_df.head(3)

,user_id,quantity,is_trial_period,purchase_date,purchase_amount
114080,8194159,1,True,2020-01-01,0.0
113381,8208663,1,True,2020-01-01,0.0
113382,8201113,1,True,2020-01-01,0.0


In [17]:
cohort_df['week'] = cohort_df['purchase_date'].dt.isocalendar().week - 1
cohort_df.tail(3)

,user_id,quantity,is_trial_period,purchase_date,purchase_amount,week
30167,8916008,1,False,2020-02-20,4.99,7
1073,9083127,1,False,2020-02-20,4.99,7
87374,8911644,1,False,2020-02-20,4.99,7


#### Будем рассчитывать текущий LTV юзера при помощи накопительного ARPU и когортного анализа

In [18]:
user_ids_list = []
n = 0
while n != 8:
    query = cohort_df[(cohort_df['week'] == n) & (cohort_df['is_trial_period'] == True)] \
                    .groupby('user_id') \
                    .agg({'purchase_amount': 'sum'}) \
                    .sort_values(['purchase_amount'], ascending=False)
    
    user_ids_list.append(query.index.tolist())
    print(f"Когорта {n+1}: {len(user_ids_list[n])}")
    n += 1

Когорта 1: 6624
Когорта 2: 8429
Когорта 3: 9135
Когорта 4: 8186
Когорта 5: 8317
Когорта 6: 9066
Когорта 7: 9038
Когорта 8: 4114


В первую когорту попадут пользователи, которые оформили пробную подписку с 01.01.2020 по 05.01.2020, таковых было 6624 человек. В первую неделю они совершили покупки, и доход от этих покупок составил 12275.40. ARPU этой когорты 1-й недели составил 12275.40 / 6624 = 1.85 .

На следующую неделю некоторые из этих же пользователей совершили платежы на общую сумму 10099.76 . Не важно, какое их количество вернулось в проект и сколько человек заплатили, размер когорты, на который мы будем делить доход, всегда 6624 пользователей.

Таким образом уже можно посчитать накопительный ARPU. Он будет равен сумме ARPU 0-й недели (неделя оформления пробной подписки) и 1-й недели. Таким же образом посчитаем оставшиеся недели.

In [19]:
n = 1
data = {'revenue':[], 'week_ARPU':[], 'cumulative_ARPU':[]}
df = pd.DataFrame(data)
week_users, revenue, week_ARPU, cumulative_ARPU = [], [], [], []

while n != 8:
    result = cohort_df[(cohort_df['week'] == n) & 
                   (cohort_df['is_trial_period'] == False) & 
                   (cohort_df['user_id'].isin(user_ids_list[0]))] \
         .agg({'user_id': 'nunique', 'purchase_amount': 'sum'})
    
    n += 1
    week_users.append(result[0])
    revenue.append(round(result[1], 2))
    
for i in revenue:
    a = (i / len(user_ids_list[0]))
    week_ARPU.append(round(a, 2))

    
df['revenue'] = revenue
df['week_ARPU'] = week_ARPU
top_row = pd.DataFrame({'revenue':[0],'week_ARPU':[0],'cumulative_ARPU':[0]})
df = pd.concat([top_row, df]).reset_index(drop = True)
revenue.insert(0, 0)

def nums_cumulative_sum(nums_list):
    return [math.fsum(nums_list[:i+1]) for i in range(len(nums_list))]

cumulative_sum = nums_cumulative_sum(revenue)

for i in cumulative_sum:
    b = (i / len(user_ids_list[0]))
    cumulative_ARPU.append(round(b, 2))

df['cumulative_ARPU'] = cumulative_ARPU

cohort1_df = df
cohort1_df.head(4)

,revenue,week_ARPU,cumulative_ARPU
0,0.00,0.00,0.00
1,12275.40,1.85,1.85
2,10099.76,1.52,3.38
3,8912.14,1.35,4.72


#### Используя этот метод, производим расчеты для всех когорт

In [20]:
data = {'cohort':[], 'week':[], 'revenue':[], 'week_ARPU':[], 'cumulative_ARPU':[]}
data_frame = pd.DataFrame(data)

def nums_cumulative_sum(nums_list):
        return [math.fsum(nums_list[:i+1]) for i in range(len(nums_list))]

cohort = 0
while cohort != 7:
    data = {'week':[], 'revenue':[], 'week_ARPU':[], 'cumulative_ARPU':[]}
    week_users, revenue, week_ARPU, cumulative_ARPU, week = [], [], [], [], []
    df = pd.DataFrame(data)
    w = 1
    while w != 8:
        result = cohort_df[(cohort_df['week'] == w) & 
                           (cohort_df['is_trial_period'] == False) & 
                           (cohort_df['user_id'].isin(user_ids_list[cohort]))] \
        .agg({'user_id': 'nunique', 'purchase_amount': 'sum'})
    
        week.append(w)
        w += 1
        week_users.append(result[0])
        revenue.append(round(result[1], 2))
    
    for i in revenue:
        a = (i / len(user_ids_list[cohort]))
        week_ARPU.append(round(a, 2))

    df['week'] = week
    df['revenue'] = revenue
    df['week_ARPU'] = week_ARPU
    top_row = pd.DataFrame({'week':[0],'revenue':[0],'week_ARPU':[0],'cumulative_ARPU':[0]})
    df = pd.concat([top_row, df]).reset_index(drop = True)
    revenue.insert(0, 0)

    cumulative_sum = nums_cumulative_sum(revenue)

    for i in cumulative_sum:
        b = (i / len(user_ids_list[cohort]))
        cumulative_ARPU.append(round(b, 2))

    df['cumulative_ARPU'] = cumulative_ARPU
    df['cohort'] = cohort + 1
    cohort += 1
    
    cohorta_df = df
    data_frame = pd.concat([data_frame, cohorta_df], ignore_index=True)

data_frame.tail(2)

,cohort,week,revenue,week_ARPU,cumulative_ARPU
54,7.0,6.0,0.00,0.00,0.00
55,7.0,7.0,7030.91,0.78,0.78


In [21]:
data_frame[data_frame['cohort'] == 2]

,cohort,week,revenue,week_ARPU,cumulative_ARPU
8,2.0,0.0,0.00,0.00,0.00
9,2.0,1.0,14.97,0.00,0.00
10,2.0,2.0,14540.86,1.73,1.73
11,2.0,3.0,11946.06,1.42,3.14
12,2.0,4.0,10404.15,1.23,4.38
13,2.0,5.0,9301.36,1.10,5.48
14,2.0,6.0,8443.08,1.00,6.48
15,2.0,7.0,3483.02,0.41,6.90


In [22]:
pivot_table = pd.pivot_table(data_frame, values='cumulative_ARPU', index=['cohort'], columns=['week'])

In [23]:
pivot_table.style.background_gradient(axis=1)

week,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000
cohort,,,,,,,,
1.000000,0.000000,1.850000,3.380000,4.720000,5.950000,7.070000,8.080000,8.330000
2.000000,0.000000,0.000000,1.730000,3.140000,4.380000,5.480000,6.480000,6.900000
3.000000,0.000000,0.000000,0.000000,1.690000,3.050000,4.240000,5.290000,5.740000
4.000000,0.000000,0.000000,0.000000,0.000000,1.600000,2.830000,3.870000,4.330000
5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.590000,2.830000,3.330000
6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.570000,2.150000
7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.780000


In [24]:
ltv = round(pivot_table[7].mean(), 2)
ltv

4.51

### Таким образом, текущий LTV юзера составляет  $4.51